In [2]:
import pandas as pd
import numpy as numpy

# Armado de sets de entrenamiento

In [3]:
train = pd.DataFrame()

In [4]:
for ventana in range(1,4):
    
    features_pageviews = pd.read_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana))
    features_conversion = pd.read_pickle("../data/features/ventana_{}_conversion.pkl".format(ventana))
    features_devicedata = pd.read_pickle("../data/features/ventana_{}_devicedata.pkl".format(ventana))
    
    label = pd.read_pickle("../data/labels/labels_trimestre_{}.pkl".format(ventana+1))

    train_actual = label.merge(features_pageviews,left_index = True, right_index = True, how = 'left')
    train_actual = train_actual.merge(features_conversion,left_index = True, right_index = True, how = 'left')
    train_actual = train_actual.merge(features_devicedata,left_index = True, right_index = True, how = 'left')
    
    train = train.append(train_actual)

In [5]:
train.head()

,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,Vistas trim No MD,Vistas trim MD
USER_ID,,,,,,,,,,,,,,,,
6574,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,74.0
2639,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,48.0
3898,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,14.0
7996,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,24.0
6111,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,32.0


### Formateo los datos

#### Columnas categoricas

In [4]:
columnas_categoricas = [  'page_mas_vista_en_mes_1',
       'page_mas_vista_en_mes_2', 'page_mas_vista_en_mes_3']

In [5]:
train[columnas_categoricas] = train[columnas_categoricas].fillna(-1)
train[columnas_categoricas] = train[columnas_categoricas].astype("int32")

#### Columnas con valores numericos

In [6]:
columnas_contar = ['cantidad_paginas_vistas_trimestre',
       'cant_paginas_vistas_en_mes_1', 'cant_paginas_vistas_en_mes_2',
       'cant_paginas_vistas_en_mes_3']

In [7]:
train[columnas_contar] = train[columnas_contar].fillna(0)
train[columnas_contar] = train[columnas_contar].astype("int64")

#### Columnas booleanas

In [8]:
columnas_booleanas = ['convirtio_en_mes_1', 'convirtio_en_mes_2', 'convirtio_en_mes_3',
       'convirtio_multiple_en_mes_1', 'convirtio_multiple_en_mes_2',
       'convirtio_multiple_en_mes_3']

In [9]:
train["convirtio"] = train["convirtio"] * 1

In [10]:
train[columnas_booleanas] = train[columnas_booleanas].fillna(0)
train[columnas_booleanas] = train[columnas_booleanas].astype("int8")

---

In [11]:
train.head(15)

,convirtio,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3
USER_ID,,,,,,,,,,,,,,
6574,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
2639,1,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
3898,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
7996,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
6111,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
2378,0,0,0,0,0,-1,-1,-1,0,1,0,0,1,0
3868,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
3861,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0
2587,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0


In [40]:
train.to_pickle("../data/train_completo.pkl")

### Armado de set prediccion

In [6]:
    
features_pageviews = pd.read_pickle("../data/features/predecir_pageviews.pkl")
features_conversion = pd.read_pickle("../data/features/predecir_conversion.pkl")
features_devicedata = pd.read_pickle("../data/features/predecir_devicedata.pkl")
    

features_pred = features_pageviews.merge(features_conversion,left_index = True, right_index = True, how = 'left')
features_pred = features_pred.merge(features_devicedata,left_index = True, right_index = True, how = 'left')

In [7]:
features_pred.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3,Vistas trim No MD,Vistas trim MD
USER_ID,,,,,,,,,,,,,,,
0,1519,527,490,502,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,477,138,140,199,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,21,12,2,7,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,1969,498,731,740,3,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,645,215,243,187,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
features_pred.to_pickle("../data/prediccion_completo.pkl")

---

## Binary encode

In [4]:
import category_encoders as ce

In [3]:
train = pd.read_pickle("../data/train_completo.pkl")

In [22]:
cat_df_flights_ce = train.drop("convirtio",axis = 1).copy()




In [23]:
encoder = ce.BinaryEncoder(cols=['page_mas_vista_en_mes_1','page_mas_vista_en_mes_2','page_mas_vista_en_mes_3'])
df_binary = encoder.fit_transform(cat_df_flights_ce)



In [24]:

df_binary.columns

Index(['cantidad_paginas_vistas_trimestre', 'cant_paginas_vistas_en_mes_1',
       'cant_paginas_vistas_en_mes_2', 'cant_paginas_vistas_en_mes_3',
       'page_mas_vista_en_mes_1_0', 'page_mas_vista_en_mes_1_1',
       'page_mas_vista_en_mes_1_2', 'page_mas_vista_en_mes_1_3',
       'page_mas_vista_en_mes_1_4', 'page_mas_vista_en_mes_1_5',
       'page_mas_vista_en_mes_1_6', 'page_mas_vista_en_mes_1_7',
       'page_mas_vista_en_mes_1_8', 'page_mas_vista_en_mes_2_0',
       'page_mas_vista_en_mes_2_1', 'page_mas_vista_en_mes_2_2',
       'page_mas_vista_en_mes_2_3', 'page_mas_vista_en_mes_2_4',
       'page_mas_vista_en_mes_2_5', 'page_mas_vista_en_mes_2_6',
       'page_mas_vista_en_mes_2_7', 'page_mas_vista_en_mes_2_8',
       'page_mas_vista_en_mes_3_0', 'page_mas_vista_en_mes_3_1',
       'page_mas_vista_en_mes_3_2', 'page_mas_vista_en_mes_3_3',
       'page_mas_vista_en_mes_3_4', 'page_mas_vista_en_mes_3_5',
       'page_mas_vista_en_mes_3_6', 'page_mas_vista_en_mes_3_7',
       '

In [42]:
df_binary.head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1_0,page_mas_vista_en_mes_1_1,page_mas_vista_en_mes_1_2,page_mas_vista_en_mes_1_3,page_mas_vista_en_mes_1_4,page_mas_vista_en_mes_1_5,...,page_mas_vista_en_mes_3_5,page_mas_vista_en_mes_3_6,page_mas_vista_en_mes_3_7,page_mas_vista_en_mes_3_8,convirtio_en_mes_1,convirtio_en_mes_2,convirtio_en_mes_3,convirtio_multiple_en_mes_1,convirtio_multiple_en_mes_2,convirtio_multiple_en_mes_3
USER_ID,,,,,,,,,,,,,,,,,,,,,
6574,1141,524,364,253,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2639,671,261,179,231,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3898,219,88,50,81,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
7996,383,95,178,110,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6111,425,208,93,124,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [44]:
df_binary["convirtio"] = train["convirtio"]

In [45]:
df_binary.to_pickle("../data/train_codificado.pkl")

In [26]:
pred = pd.read_pickle("../data/prediccion_completo.pkl")

In [27]:
df_binary_pred = encoder.transform(pred)



In [41]:
df_binary_pred.to_pickle("../data/prediccion_codificado.pkl")